In [1]:
!pip install antigranular &> /dev/null

In [2]:
import antigranular as ag
session = ag.login(<client_id>,<client_secret>, competition = "Fraud Detection Hackathon with Privacy Village #DPD25FEST")

Dataset "Transaction Fraud Hackathon Dataset" loaded to the kernel as transaction_fraud_hackathon_dataset
Key Name                       Value Type     
---------------------------------------------
test_x                         DataFrame      
train_x                        PrivateDataFrame
train_y                        PrivateDataFrame

Connected to Antigranular server session id: fe45a754-18dc-4bac-98b5-24d9fac2185f, the session will time out if idle for 25 minutes
Cell magic '%%ag' registered successfully, use `%%ag` in a notebook cell to execute your python code on Antigranular private python server
🚀 Everything's set up and ready to roll!


In [ ]:
%%ag
x_train = transaction_fraud_hackathon_dataset["train_x"]
y_train = transaction_fraud_hackathon_dataset["train_y"]
x_test = transaction_fraud_hackathon_dataset["test_x"]

In [ ]:
%%ag

#Sequential model nn with conv
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from op_tensorflow import PrivateKerasModel, PrivateDataLoader
import op_tensorflow
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv1D,Dropout,Flatten,BatchNormalization
from op_tensorflow import PrivateKerasModel, PrivateDataLoader
import op_pandas
from op_pandas import standard_scaler, PrivateDataFrame, train_test_split

concat_df = x_train.join(y_train)
train_data, test_data = op_pandas.train_test_split(concat_df, test_size=0.0001, random_state=42)

train_y = train_data['fraud'].copy()
train_data.drop(columns=['fraud'], inplace=True)

test_y = test_data['fraud'].copy()
test_data.drop(columns=['fraud'], inplace=True)


batch_size = 32
epochs = 15
noise_multiplier = 0.3
target_delta = 1


model = Sequential([
    Conv1D(128,6, activation='relu', input_shape=(9, 1)),
    BatchNormalization(),
    Conv1D(128,4, activation='relu',padding='same'),
    BatchNormalization(),
    Flatten(),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dense(1, activation='sigmoid')
])


dp_model = PrivateKerasModel(
    model=model,
    l2_norm_clip=1,
    noise_multiplier=noise_multiplier
)


optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# Model compile
dp_model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=['accuracy']
)

optimizer = tf.keras.optimizers.Nadam(learning_rate=0.01)

# Model compile
dp_model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=['accuracy']
)
# Data loader
data_loader = PrivateDataLoader(
    feature_df=train_data,
    label_df=train_y,
    batch_size=batch_size
)


dp_model.fit(
    x=data_loader,
    epochs=epochs,
    target_delta=target_delta,
    #validation_data=(test_data, test_y)
)



In [ ]:
%%ag
from op_pandas import standard_scaler, PrivateDataFrame
test=PrivateDataFrame(x_test).copy()
y_pred = dp_model.predict(test, label_columns=["output"])
# Note that the predictions are a float scalar
# so we scale it
def f(x: float) -> int:
  if x > 0.5:
    return 1
  else:
    return 0

y_pred["output"] = y_pred["output"].map(f, output_categories=[0, 1])
result = submit_predictions(y_pred)

In [ ]:
session.privacy_odometer()